In [1]:
import yaml
import numpy as np
import pandas as ps
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from pathlib import Path

import matplotlib.pyplot as plt
from skimage.io import imshow

from albumentations.core.serialization import from_dict
from catalyst.dl.runner import SupervisedRunner
import catalyst.dl.callbacks as cbks

from dataset import SteelDataset
from models import UNetResNet
from metrics import MulticlassDiceMetricCallback
from losses import MulticlassDiceLoss

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
config_file = Path('.') / 'configs' / 'simple_unet.yaml'

with open(config_file, 'r') as fuck:
    config = yaml.safe_load(fuck)

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DATA_FOLDER = Path('..') / 'data'
RANDOM_STATE = np.random.RandomState(seed=2019)
NUM_WORKERS = 12
BATCH_SIZE = 16
NUM_CLASSES = 4
validation_pcnt = 0.1

In [3]:
def load_transforms(transforms_dict: dict):
    if transforms_dict is not None:
        return from_dict(transforms_dict)
    return None


train_transforms = load_transforms(config['train']['transformations']['train'])
validation_transforms = load_transforms(config['train']['transformations']['validation'])

In [4]:
train_df = ps.read_csv(config['train']['file'])
train_images_folder = config['train']['folder']

print(f'train images folder - {train_images_folder}')
train_df.head()

train images folder - /home/dmdr/Documents/Code/Python/kaggle/severstal_steel_defect_detection/data/train_images


,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,NaN
2,0002cc93b.jpg_3,NaN
3,0002cc93b.jpg_4,NaN
4,00031f466.jpg_1,NaN


In [5]:
train_df = train_df[train_df['EncodedPixels'].notnull()].reset_index(drop=True)
train_df['Image'] = train_df['ImageId_ClassId'].apply(lambda img_cls: img_cls.rsplit('_', 1)[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda img_cls: int(img_cls.rsplit('_', 1)[1]) - 1)
train_df = train_df.sort_values(['Image', 'ClassId'])
# train_df = train_df.drop(columns=['ImageId_ClassId'])

print(train_df.dtypes)

train_df.head()

ImageId_ClassId    object
EncodedPixels      object
Image              object
ClassId             int64
dtype: object


,ImageId_ClassId,EncodedPixels,Image,ClassId
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,0002cc93b.jpg,0
1,0007a71bf.jpg_3,18661 28 18863 82 19091 110 19347 110 19603 11...,0007a71bf.jpg,2
2,000a4bcdd.jpg_1,37607 3 37858 8 38108 14 38359 20 38610 25 388...,000a4bcdd.jpg,0
3,000f6bf48.jpg_4,131973 1 132228 4 132483 6 132738 8 132993 11 ...,000f6bf48.jpg,3
4,0014fce06.jpg_3,229501 11 229741 33 229981 55 230221 77 230468...,0014fce06.jpg,2


In [6]:
train_df['ClassId'].value_counts()

2    5150
0     897
3     801
1     247
Name: ClassId, dtype: int64

In [7]:
def combine_masks(df):
    masks = [[]] * 4
    for idx in df.index:
        masks[df.at[idx, 'ClassId']] = np.array(list(map(int, df.at[idx, 'EncodedPixels'].split())))
    return ps.Series([masks], ['RLEs'])
    

train_set = train_df.groupby('Image').apply(combine_masks).reset_index()

for cls_idx in range(4):
    cls_name = f'IsPresent{cls_idx}'
    train_set[cls_name] = False
    for idx in train_set.index:
        train_set.at[idx, cls_name] = len(train_set.at[idx, 'RLEs'][cls_idx]) > 0


print(train_set.shape)
train_set.head()

(6666, 6)


,Image,RLEs,IsPresent0,IsPresent1,IsPresent2,IsPresent3
0,0002cc93b.jpg,"[[29102, 12, 29346, 24, 29602, 24, 29858, 24, ...",True,False,False,False
1,0007a71bf.jpg,"[[], [], [18661, 28, 18863, 82, 19091, 110, 19...",False,False,True,False
2,000a4bcdd.jpg,"[[37607, 3, 37858, 8, 38108, 14, 38359, 20, 38...",True,False,False,False
3,000f6bf48.jpg,"[[], [], [], [131973, 1, 132228, 4, 132483, 6,...",False,False,False,True
4,0014fce06.jpg,"[[], [], [229501, 11, 229741, 33, 229981, 55, ...",False,False,True,False


In [8]:
validation_set = train_set.sample(int(train_set.shape[0] * validation_pcnt),
                                  random_state=RANDOM_STATE)
print(validation_set.shape)
validation_set.head()

(666, 6)


,Image,RLEs,IsPresent0,IsPresent1,IsPresent2,IsPresent3
5524,d2c14891a.jpg,"[[], [], [285887, 24, 286117, 51, 286367, 59, ...",False,False,True,False
2292,5a3db6845.jpg,"[[], [], [172782, 19, 173002, 55, 173221, 92, ...",False,False,True,False
3179,7a5fae002.jpg,"[[], [], [275937, 32, 276129, 96, 276321, 160,...",False,False,True,False
600,17c821b3f.jpg,"[[131870, 32, 132097, 66, 132353, 71, 132609, ...",True,False,False,False
2076,5132041da.jpg,"[[], [], [366032, 12, 366286, 15, 366539, 18, ...",False,False,True,False


In [9]:
def get_loaders() -> dict:
    train_dataset = SteelDataset(
        images=train_set['Image'].values, 
        rles=train_set['RLEs'].values,
        folder=DATA_FOLDER / 'train_images',
        transforms=train_transforms
    )
    validation_dataset = SteelDataset(
        images=validation_set['Image'].values, 
        rles=validation_set['RLEs'].values,
        folder=DATA_FOLDER / 'train_images',
        transforms=validation_transforms
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS
    )

    validation_loader = DataLoader(
        validation_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS
    )
    
    print(f'Number of batches in train - {len(train_loader)}, '
          f'Number of batches in validation - {len(validation_loader)}')
    
    return {'train': train_loader, 'valid': validation_loader}

In [10]:
from catalyst.contrib.models.segmentation import Unet

model = Unet(num_classes=5, in_channels=1, num_channels=64, num_blocks=4).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5, factor=0.2, verbose=True)

multiclass_dice_loss = MulticlassDiceLoss([1, 2, 3, 4], from_logits=True, weight=None, reduction='elementwise_mean')
cce_loss = CCE(ignore_index=5)
alpha = 0.7
joint_loss = JointLoss(cce_loss, multiclass_dice_loss, alpha, 1 - alpha)

loaders = get_loaders()

Number of batches in train - 417, Number of batches in validation - 42


In [11]:
runner = SupervisedRunner(device=DEVICE)
runner.train(
    model=model,
    criterion=loss_function,
    optimizer=optimizer,
    loaders=loaders,
    logdir='logs',
    callbacks=[
        MulticlassDiceMetricCallback(class_names=['0', '1', '2', '3', '4']),
        cbks.CheckpointCallback(save_n_best=3),
#         cbks.EarlyStoppingCallback(5, metric='dice')
    ],
    scheduler=scheduler,
    verbose=True,
    minimize_metric=False,
    num_epochs=50
)

0/50 * Epoch (train): 100% 417/417 [03:50<00:00,  2.20s/it, 0=1.000, 1=0.000, 2=0.001, 3=0.414, 4=0.063, _timers/_fps=5.985, loss=0.542] 
0/50 * Epoch (valid): 100% 42/42 [00:07<00:00,  6.68it/s, 0=1.000, 1=0.008, 2=0.000, 3=0.366, 4=0.000, _timers/_fps=3680.425, loss=0.332]
[2019-08-11 01:26:18,072] 
0/50 * Epoch 0 (train): 0=0.8343 | 1=0.0763 | 2=0.0005 | 3=0.4315 | 4=0.1542 | _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=47.6921 | _timers/batch_time=0.3594 | _timers/data_time=0.3449 | _timers/model_time=0.0145 | loss=0.5361
0/50 * Epoch 0 (valid): 0=1.0000 | 1=0.0099 | 2=0.0002 | 3=0.4513 | 4=0.4719 | _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=3324.3313 | _timers/batch_time=0.0106 | _timers/data_time=0.0083 | _timers/model_time=0.0022 | loss=0.4485
1/50 * Epoch (train): 100% 417/417 [03:37<00:00,  1.98it/s, 0=1.000, 1=0.007, 2=0.004, 3=0.628, 4=0.000, _timers/_fps=46.299, loss=0.481]
1/50 * Epoch (valid): 100% 42/42 [00:07<00:00,  6.75it/s, 0=1.000, 1=0.048, 2=0